In [11]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, timedelta, datetime

######## Select ticker, initial date, final date, initial amount, periodic amount to invest and periodicity in week days
ticker = "AMD"
initial_date = date(2023,1,1)
final_date = date.today()
final_date = date(2025,1,22)
df = yf.download(ticker,initial_date,final_date, progress=False)
df.columns = df.columns.droplevel(1)
##############################


df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['STD_20'] = df['Close'].rolling(window=20).std()
df['BB_upper'] = df['SMA_20'] + 2 * df['STD_20']
df['BB_lower'] = df['SMA_20'] - 2 * df['STD_20']

df['SMA_200'] = df['Close'].rolling(window=200).mean()

df['Volume_MA20'] = df['Volume'].rolling(window=20).mean()

df['Breakout_Up'] = (df['Close'] > df['BB_upper']) & (df['Volume'] > df['Volume_MA20'])
df['Breakout_Down'] = (df['Close'] < df['BB_lower']) & (df['Volume'] > df['Volume_MA20'])

df['Volume_Color'] = df.apply(lambda row: 'green' if row['Close'] >= row['Open'] else 'red', axis=1)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05, subplot_titles=('OHLC con Bandas de Bollinger y SMA 200', 'Volumen'),
                    row_width=[0.2, 0.7])  # Volumen más pequeño en la segunda fila

fig.add_trace(go.Candlestick(
    x=df.index, open=df["Open"], high=df["High"], low=df["Low"], close=df["Close"],
    name="OHLC"
), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['BB_upper'], line=dict(color='blue', width=1), name='Upper Band'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='orange', width=1), name='20-Day MA'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['BB_lower'], line=dict(color='blue', width=1), name='Lower Band'), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], line=dict(color='purple', width=2, dash='dot'), name='SMA 200'), row=1, col=1)

'''
for i in df.index:
    if df.loc[i, 'Breakout_Up']:  # Ruptura alcista
        #fig.add_trace(go.Scatter(x=[i, i], y=[df['Low'].min(), df['High'].max()], mode="lines",
                                 line=dict(color="green", width=1), name="Ruptura Alcista"),
                      row=1, col=1)
    if df.loc[i, 'Breakout_Down']:  # Ruptura bajista
        #fig.add_trace(go.Scatter(x=[i, i], y=[df['Low'].min(), df['High'].max()], mode="lines",
                                 line=dict(color="red", width=1), name="Ruptura Bajista"),
                      row=1, col=1)
'''

for i in range(len(df)):
    fig.add_trace(go.Bar(
        x=[df.index[i]], y=[df['Volume'].iloc[i]],
        marker=dict(color=df['Volume_Color'].iloc[i]),
        showlegend=False  # No repetir leyenda en cada barra
    ), row=2, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['Volume_MA20'], line=dict(color='orange', width=2),
                         name='Volume MA 20'), row=2, col=1)

fig.update_layout(title=f"Análisis de {ticker} con Bandas de Bollinger, SMA 200 y Volumen",
                  xaxis_rangeslider_visible=False,
                  showlegend=True)

fig.show()
# RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'], window=14)

# MACD
def calculate_macd(data, fast_length=12, slow_length=26, signal_smooth=9):
    ema_fast = data.ewm(span=fast_length, adjust=False).mean()
    ema_slow = data.ewm(span=slow_length, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=signal_smooth, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

df['MACD'], df['Signal'], df['MACD_Histogram'] = calculate_macd(df['Close'])

# ADX
def calculate_adx(df, window=14):
    high = df['High']
    low = df['Low']
    close = df['Close']

    # Calcular los movimientos de alta y baja
    up_move = high.diff()
    down_move = low.diff()

    # Calcular los valores de +DM y -DM
    plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0), index=df.index)
    minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0), index=df.index)

    # Calcular el True Range (TR) y el Average True Range (ATR)
    tr = pd.concat([high - low, abs(high - close.shift()), abs(low - close.shift())], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()

    # Calcular el +DI y -DI
    plus_di = 100 * (plus_dm.rolling(window=window).sum() / atr)
    minus_di = 100 * (minus_dm.rolling(window=window).sum() / atr)

    # Calcular el ADX
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.rolling(window=window).mean()

    return adx, plus_di, minus_di

df['ADX'], df['+DI'], df['-DI'] = calculate_adx(df)

# Mostrar resultados
print(df[['Close', 'RSI', 'MACD', 'Signal', 'ADX', '+DI', '-DI']].tail())

Price            Close        RSI      MACD    Signal        ADX         +DI  \
Date                                                                           
2025-01-14  116.089996  45.888244 -4.186415 -4.071078  17.295520  200.887011   
2025-01-15  119.959999  43.629880 -4.007550 -4.058372  18.419708  106.264700   
2025-01-16  118.440002  39.172423 -3.942996 -4.035297  20.004433  106.388133   
2025-01-17  121.459999  45.268143 -3.606574 -3.949552  21.911507  102.888531   
2025-01-21  122.279999  46.243219 -3.236482 -3.806938  23.226503  128.232494   

Price              -DI  
Date                    
2025-01-14  235.254976  
2025-01-15  314.849120  
2025-01-16  236.236875  
2025-01-17  271.373598  
2025-01-21  277.110116  


### 1. RSI (Relative Strength Index)

El RSI mide la velocidad y el cambio de los movimientos de precios, con valores entre 0 y 100. Los valores por encima de 70 indican sobrecompra y los valores por debajo de 30 indican sobreventa.

### 2. MACD (Moving Average Convergence Divergence)

El MACD es un indicador de tendencia que muestra la relación entre dos medias móviles exponenciales (EMA).

### 3. ADX (Average Directional Index)

El ADX mide la fuerza de la tendencia. Un valor por encima de 25 indica una tendencia fuerte, mientras que un valor por debajo de 20 indica una tendencia débil o sin tendencia.
